In [1]:
import os
from ROOT import *
from ML_Clasification import LQ_Log_Reg

Welcome to JupyROOT 6.26/06


2022-09-20 08:37:53.512596: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-20 08:37:53.512625: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ImportError: cannot import name 'DrawDiscrtiminator' from 'bsm_ml' (/home/crisfer2694/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/bsm_ml.py)

In [ ]:
folder_out=os.path.join(
    os.path.dirname(os.getcwd()),
    "03_delphes_analysis"
)
csv_files_path=os.path.join(folder_out,"Selected_Events")
channel="b_b_tau_tau_hadronic"
mass=1500
print(csv_files_path)


In [ ]:
modelo=LQ_Log_Reg(mass,channel,csv_files_path)
logreg_model=modelo.get_log_reg_model()

In [ ]:
images_folder=os.path.join(f"MLQ_{mass}",f"{channel}")
os.system(f"rm -rf '{images_folder}'")
os.system(f"mkdir -p '{images_folder}'")
def mapping(name):
    return DrawDiscrtiminator(name,[modelo.channel],csv_files_path,images_folder,logreg_model,modelo.features)
histograms=dict(map(mapping,modelo.signal_names+modelo.bkg_names))

In [ ]:
from IPython.display import Image, display
import os
listOfImageNames=[]
files=next(os.walk(images_folder))[2]
for file in files:
    if ".png" in file:
        listOfImageNames+=[file]
listOfImageNames.sort()
for imageName in listOfImageNames:
    display(Image(filename=os.path.join(os.getcwd(),images_folder,imageName)))
    os.remove(os.path.join(os.getcwd(),images_folder,imageName))

In [ ]:
histograms

In [ ]:
threshold=0.8

In [ ]:
integral=0
for channel in channels:
    h=histograms[name][channel]
    gbin1=h.FindBin(threshold)
    gbin2=h.GetNbinsX()
    integral+=h.Integral(gbin1,gbin2)
integral

In [ ]:
actual_signal_yield=0
for name in signal_names:
    integral=0
    for channel in channels:
        h=histograms[name][channel]
        gbin1=h.FindBin(threshold)
        gbin2=h.GetNbinsX()
        integral+=h.Integral(gbin1,gbin2)
    integral
    actual_signal_yield+=integral
print(signal_names,actual_signal_yield)

In [ ]:
actual_bkg_yield=0
for name in bkg_names:
    integral=0
    for channel in channels:
        h=histograms[name][channel]
        gbin1=h.FindBin(threshold)
        gbin2=h.GetNbinsX()
        integral+=h.Integral(gbin1,gbin2)
    actual_bkg_yield+=integral
print(bkg_names,actual_bkg_yield)

In [ ]:
significance=actual_signal_yield/math.sqrt(actual_signal_yield+actual_bkg_yield)
significance

In [ ]:
def get_yield_by_names(hists,names,threshold):
    signal_yield=0
    for name in names:
        integral=0
        for channel in channels:
            h=histograms[name][channel]
            gbin1=h.FindBin(threshold)
            gbin2=h.GetNbinsX()
            integral+=h.Integral(gbin1,gbin2)
        integral
        signal_yield+=integral
    return signal_yield

In [ ]:
def get_sig_by_threshold(histos,threshold=0.5):
    actual_signal_yield=get_yield_by_names(histos,signal_names,threshold)
    actual_bkg_yield=get_yield_by_names(histos,bkg_names,threshold)
    return (actual_signal_yield)/math.sqrt((actual_signal_yield)+(actual_bkg_yield)+1e-6)

In [ ]:
get_sig_by_threshold(histograms,0.99)

In [ ]:
x=[]
y1=[]
y2=[]
n=1000
for i in range(n+1):
    x+=[i*(1./float(n))]
    y1+=[get_yield_by_names(histograms,bkg_names,x[i])]
    y2+=[get_yield_by_names(histograms,signal_names,x[i])]
x  = np.array(  x )
y1  = np.array(  y1 )
y2  = np.array(  y2 )

c2=TCanvas("c2","",800,600)
c2.SetLogy()
c2.SetGrid()
graph1=TGraph(n, x, y1)
graph1.SetTitle(f"N-Events (137/fb) (m_{{LQ}}={mass}GeV);threshold; Events")
graph1.GetXaxis().SetLimits(0,1)
graph1.GetHistogram().SetMinimum(1)
graph1.GetHistogram().SetMaximum(3e5)
graph1.Draw("ALP")
graph1.SetLineColor(ROOT.kBlue)
graph1.SetLineWidth(2)

graph2=TGraph(n, x, y2)
graph2.Draw("l-same")
graph2.SetLineColor(ROOT.kRed)
graph2.SetLineWidth(2)
c2.Draw()

In [ ]:
x=[]
y=[]
n=1000
for i in range(n+1):
    x+=[i*(1./float(n))]
    y+=[get_yield_by_names(histograms,signal_names,x[i])]
x  = np.array(  x )
y  = np.array(  y )/y[0]

c2=TCanvas("c2","",800,600)
c2.SetGrid()
graph=TGraph(n, x, y)
graph.SetTitle(f"ML-cutflow (m_{{LQ}}={mass}GeV);threshold;signal efficiency")
graph.GetXaxis().SetLimits(0,1)
graph.Draw("ALP")
graph.SetLineColor(ROOT.kBlue)
graph.SetLineWidth(2)
c2.Draw()

In [ ]:
x=[]
y=[]
n=1000
for i in range(n+1):
    x+=[i*(1./float(n))]
    y+=[get_yield_by_names(histograms,bkg_names,x[i])]
x  = np.array(  x )
y  = np.array(  y )/y[0]

c2=TCanvas("c2","",800,600)
c2.SetLogy()
c2.SetGrid()
graph=TGraph(n, x, y)
graph.SetTitle(f"ML-cutflow (m_{{LQ}}={mass}GeV);threshold;background efficiency")
graph.GetXaxis().SetLimits(0,1)
graph.GetHistogram().SetMinimum(1e-5)
graph.GetHistogram().SetMaximum(1)
graph.Draw("ALP")
graph.SetLineColor(ROOT.kBlue)
graph.SetLineWidth(2)
c2.Draw()

In [ ]:
x=[]
y=[]
n=1000
for i in range(n+1):
    x+=[i*(1./float(n))]
    y+=[get_sig_by_threshold(histograms,x[i])]
x  = np.array(  x )
y  = np.array(  y )
c2=TCanvas("c2","",800,600)
c2.SetGrid()
graph=TGraph(n, x, y)
graph.SetTitle("Logistic Regression significance;threshold;significance(#sigma )[137/fb]")
graph.GetXaxis().SetLimits(0,1)
graph.Draw("ALP")
graph.SetLineColor(ROOT.kBlue)
graph.SetLineWidth(2)
c2.Draw()

In [ ]:
x=[]
y=[]
n=100
for i in range(n+1):
    x+=[i*(1./float(n))]
    y+=[get_sig_by_threshold(histograms,x[i])]
x  = np.array(  x  )
y  = np.array(  y  )
print(y)

In [ ]:
histograms

In [ ]:
from ROOT import TFile
import os
f = TFile(os.path.join(images_folder,"Logistic_Regresion.root"),"RECREATE")
for signal in histograms:
    h=histograms[signal][channel]
    h.Write(h.GetName().removesuffix(channel))
f.ls()

In [ ]:

h.GetBinContent(h.GetNbinsX())

In [ ]:
h.GetNbinsX()

In [ ]:
h.GetMinimumBin()

In [ ]:
for bin_ in range(h.GetNbinsX()):
    continue